# 01 Data preprocessing

- 이 단계에서는 데이터 셋을 다운받고, Amazon Translate을 사용해 한국어로 번역한 후 Amazon S3에 저장하는 과정을 수행합니다.

### 데이터셋 불러오기

In [ ]:
from datasets import load_dataset

# 데이터셋 불러오기
dataset = load_dataset("b-mc2/sql-create-context")

# 'train' split에서 첫 번째 샘플 출력
print("전문: ", dataset['train'][0])
print("#######구분해서 보기######")
sample = dataset['train'][0]
print("Answer:", sample['answer'])
print("Question:", sample['question'])
print("Context:", sample['context'])

### Amazon Translate으로 데이터 한국어로 번역하기

In [ ]:
from datasets import load_dataset
import boto3

# AWS 클라이언트 생성
translate = boto3.client(service_name='translate', region_name='us-east-1')

# 데이터셋 불러오기
dataset = load_dataset("b-mc2/sql-create-context")

# 'question' 열 추출 및 번역
translated_dataset = dataset.map(
    lambda sample: {
        'answer': sample['answer'],
        'question': [
            translate.translate_text(Text=question, SourceLanguageCode="en", TargetLanguageCode="ko").get('TranslatedText')
            for question in sample['question']
        ],
        'context': sample['context']
    },
    batched=True,
    desc="Translating questions to Korean"
)

# 번역된 데이터셋 저장
translated_dataset.save_to_disk('data/translated')

#### 번역 완료 확인하기

In [ ]:
from datasets import load_dataset
from IPython.display import display

# 번역된 데이터셋 불러오기
translated_dataset = translated_dataset.load_from_disk('data/translated')

# 데이터셋 정보 출력
print(translated_dataset)

# 샘플 5개 출력
for sample in translated_dataset['train']['question'][:5]:
    display(sample)

### 데이터 셋 전처리

- 대화형으로 데이터셋 변경

In [ ]:
from datasets import load_dataset, DatasetDict

system_message = """You are a powerful text-to-SQL model. Your job is to answer questions about a database."""

def create_conversation(record):
    sample = {"messages": [
        {"role": "system", "content": system_message + f"""You can use the following table schema for context: {record["context"]}"""},
        {"role": "user", "content": f"""Return the SQL query that answers the following question: {record["question"]}"""},
        {"role" : "assistant", "content": f"""{record["answer"]}"""}
    ]}
    return sample

dataset = translated_dataset
dataset = dataset.map(create_conversation, batched=False).remove_columns(['answer', 'question', 'context'])

# 먼저 전체 데이터셋에서 test set을 분리
train_val_test_split = dataset["train"].train_test_split(test_size=0.1, seed=42)

# 남은 데이터에서 train과 validation을 분리
train_val_split = train_val_test_split["train"].train_test_split(test_size=0.1, seed=42)

# 최종 데이터셋 구성
training_data = train_val_split["train"]
validation_data = train_val_split["test"]
test_data = train_val_test_split["test"]

# 각 데이터셋을 JSON 파일로 저장
training_data.to_json("../datasets/ko_train_dataset.json", orient="records", force_ascii=False)
validation_data.to_json("../datasets/ko_validation_dataset.json", orient="records", force_ascii=False)
test_data.to_json("../datasets/ko_test_dataset.json", orient="records", force_ascii=False)

- json으로 타입 변경

In [ ]:
import json
from IPython.display import display

train_data = []

# Load the JSON data line by line
with open('../datasets/ko_train_dataset.json', 'r') as f:
    for line in f:
        try:
            train_data.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON line: {line}")

# Display the first 5 examples
for example in train_data[:5]:
    display(example)

In [ ]:
import json
from IPython.display import display

train_data = []

# Load the JSON data line by line
with open('../datasets/ko_test_dataset.json', 'r') as f:
    for line in f:
        try:
            train_data.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON line: {line}")

# Display the first 5 examples
for example in train_data[:5]:
    display(example)

In [ ]:
import json
from IPython.display import display

train_data = []

# Load the JSON data line by line
with open('../datasets/ko_validation_dataset.json', 'r') as f:
    for line in f:
        try:
            train_data.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON line: {line}")

# Display the first 5 examples
for example in train_data[:5]:
    display(example)

### S3에 데이터 업로드하기

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()

sagemaker_session_bucket = sess.default_bucket()
    
# S3 클라이언트 생성
s3_client = boto3.client('s3')

# 파일 경로와 S3 업로드 경로 설정
train_local_file_path = '../datasets/ko_train_dataset.json'
train_s3_upload_path = 'data/train/ko_train_dataset.json'

validation_local_file_path = '../datasets/ko_validation_dataset.json'
validation_s3_upload_path = 'data/validation/ko_validation_dataset.json'

test_local_file_path = '../datasets/ko_test_dataset.json'
test_s3_upload_path = 'data/test/ko_test_dataset.json'

# 파일 업로드 함수
def upload_file(local_path, s3_path):
    try:
        s3_client.upload_file(local_path, sagemaker_session_bucket, s3_path)
        print(f"파일 {local_path}가 {sagemaker_session_bucket}/{s3_path}에 업로드되었습니다.")
    except Exception as e:
        print(f"파일 {local_path} 업로드 실패: {e}")

# 파일 업로드
upload_file(train_local_file_path, train_s3_upload_path)
upload_file(validation_local_file_path, validation_s3_upload_path)
upload_file(test_local_file_path, test_s3_upload_path)